# CA11: Advanced Model-Based RL and World Models

## Deep Reinforcement Learning - Session 11

**Advanced Model-Based Reinforcement Learning: World Models, Planning in Latent Space, and Modern Approaches**

This notebook demonstrates cutting-edge model-based reinforcement learning techniques using modular implementations. The code has been restructured into separate modules for better organization and reusability.

### Learning Objectives:
1. Understand world models and latent state representations
2. Implement variational autoencoders for state modeling
3. Master planning in latent space with learned dynamics
4. Explore uncertainty quantification in model-based RL
5. Implement Dreamer-style world model learning
6. Apply modular design principles to complex RL systems

### Modular Structure:
- **world_models/**: VAE, dynamics, reward models, RSSM
- **agents/**: Latent actor-critic, Dreamer agent
- **environments/**: Custom continuous control tasks
- **utils/**: Data collection and visualization
- **experiments/**: Complete training scripts

---

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from models.vae import VariationalAutoencoder
from models.dynamics import LatentDynamicsModel
from models.reward_model import RewardModel
from models.world_model import WorldModel
from models.rssm import RecurrentStateSpaceModel
from models.trainers import WorldModelTrainer, RSSMTrainer

from agents.latent_actor import LatentActor
from agents.latent_critic import LatentCritic
from agents.dreamer_agent import DreamerAgent

from environments.continuous_cartpole import ContinuousCartPole
from environments.continuous_pendulum import ContinuousPendulum
from environments.sequence_environment import SequenceEnvironment

from utils.data_collection import collect_world_model_data, collect_sequence_data
from utils.visualization import plot_world_model_training, plot_rssm_training

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Advanced Model-Based RL Environment Setup")
print(f"Device: {device}")
print(f"PyTorch version: {torch.__version__}")

plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (15, 10)
colors = sns.color_palette("husl", 8)
sns.set_palette(colors)

print("✅ Modular environment setup complete!")
print("🌟 Ready for advanced model-based reinforcement learning!")


# Section 1: World Models and Latent Representations

## 1.1 Understanding the Modular Architecture

The world model consists of three main components:
- **VAE**: Learns compressed latent representations of observations
- **Dynamics Model**: Predicts next latent states given current state and action
- **Reward Model**: Predicts rewards in latent space

Let's explore each component:

In [ ]:
env = ContinuousCartPole()
print(f"Environment: {env.name}")
print(f"Observation space: {env.observation_space.shape}")
print(f"Action space: {env.action_space.shape}")

sample_data = collect_world_model_data(env, steps=1000, episodes=5)
print(f"Collected {len(sample_data['observations'])} transitions")
print(f"Sample observation shape: {sample_data['observations'][0].shape}")
print(f"Sample action shape: {sample_data['actions'][0].shape}")


In [ ]:
obs_dim = env.observation_space.shape[0]
latent_dim = 32
vae_hidden_dims = [128, 64]

vae = VariationalAutoencoder(obs_dim, latent_dim, vae_hidden_dims).to(device)
print(f"VAE Architecture:")
print(f"Input dim: {obs_dim}, Latent dim: {latent_dim}")
print(f"Hidden dims: {vae_hidden_dims}")

test_obs = torch.randn(10, obs_dim).to(device)
recon_obs, mu, log_var, z = vae(test_obs)
print(f"Reconstruction shape: {recon_obs.shape}")
print(f"Latent shape: {z.shape}")
print(f"KL divergence: {vae.kl_divergence(mu, log_var):.4f}")


In [ ]:
action_dim = env.action_space.shape[0]
dynamics_hidden_dims = [128, 64]
reward_hidden_dims = [64, 32]

dynamics = LatentDynamicsModel(latent_dim, action_dim, dynamics_hidden_dims, stochastic=True).to(device)
reward_model = RewardModel(latent_dim, action_dim, reward_hidden_dims).to(device)

world_model = WorldModel(vae, dynamics, reward_model).to(device)
print(f"World Model created with:")
print(f"- VAE: {obs_dim} -> {latent_dim}")
print(f"- Dynamics: {latent_dim} + {action_dim} -> {latent_dim}")
print(f"- Reward: {latent_dim} + {action_dim} -> 1")

test_obs = torch.randn(5, obs_dim).to(device)
test_action = torch.randn(5, action_dim).to(device)

next_obs_pred, reward_pred = world_model.predict_next_state_and_reward(test_obs, test_action)
print(f"Prediction shapes: obs={next_obs_pred.shape}, reward={reward_pred.shape}")


In [ ]:
trainer = WorldModelTrainer(world_model, learning_rate=1e-3, device=device)

train_data = {
    'observations': torch.FloatTensor(sample_data['observations']).to(device),
    'actions': torch.FloatTensor(sample_data['actions']).to(device),
    'rewards': torch.FloatTensor(sample_data['rewards']).to(device),
    'next_observations': torch.FloatTensor(sample_data['next_observations']).to(device)
}

print("Training world model for 500 steps...")
for step in tqdm(range(500)):
    batch_size = 64
    indices = torch.randperm(len(train_data['observations']))[:batch_size]
    batch = {k: v[indices] for k, v in train_data.items()}
    losses = trainer.train_step(batch)

print("Training completed!")
plot_world_model_training(trainer, "World Model Training Demo")


# Section 2: Recurrent State Space Models (RSSM)

## 2.1 Temporal World Modeling

RSSM extends world models with recurrent networks to capture temporal dependencies:

In [ ]:
seq_env = SequenceEnvironment(memory_size=5)
print(f"Sequence Environment: {seq_env.name}")
print(f"Observation space: {seq_env.observation_space.shape}")

seq_data = collect_sequence_data(seq_env, episodes=50, episode_length=20)
print(f"Collected {len(seq_data)} episodes")
print(f"Sample episode length: {len(seq_data[0]['observations'])}")


In [ ]:
obs_dim = seq_env.observation_space.shape[0]
action_dim = seq_env.action_space.shape[0]
state_dim = 32
hidden_dim = 128

rssm = RecurrentStateSpaceModel(obs_dim, action_dim, state_dim, hidden_dim).to(device)
print(f"RSSM Architecture:")
print(f"Observation dim: {obs_dim}, Action dim: {action_dim}")
print(f"State dim: {state_dim}, Hidden dim: {hidden_dim}")

test_obs = torch.randn(1, 1, obs_dim).to(device)
test_action = torch.randn(1, 1, action_dim).to(device)
hidden = torch.zeros(1, hidden_dim).to(device)

next_obs_pred, reward_pred, next_hidden = rssm.imagine(test_obs, test_action, hidden)
print(f"Imagination shapes: obs={next_obs_pred.shape}, reward={reward_pred.shape}, hidden={next_hidden.shape}")


In [ ]:
rssm_trainer = RSSMTrainer(rssm, learning_rate=1e-3, device=device)

print("Training RSSM for 500 steps...")
for step in tqdm(range(500)):
    episode_idx = np.random.randint(len(seq_data))
    episode = seq_data[episode_idx]
    
    seq_len = min(15, len(episode['observations']))
    start_idx = np.random.randint(max(1, len(episode['observations']) - seq_len))
    
    batch = {
        'observations': torch.FloatTensor(episode['observations'][start_idx:start_idx+seq_len]).unsqueeze(0).to(device),
        'actions': torch.FloatTensor(episode['actions'][start_idx:start_idx+seq_len]).unsqueeze(0).to(device),
        'rewards': torch.FloatTensor(episode['rewards'][start_idx:start_idx+seq_len]).unsqueeze(0).to(device)
    }
    
    losses = rssm_trainer.train_step(batch)

print("RSSM training completed!")
plot_rssm_training(rssm_trainer, "RSSM Training Demo")


# Section 3: Dreamer Agent - Planning in Latent Space

## 3.1 Complete Model-Based RL

The Dreamer agent combines world models with actor-critic methods in latent space:

In [ ]:
actor = LatentActor(latent_dim, action_dim, hidden_dims=[128, 64]).to(device)
critic = LatentCritic(latent_dim, hidden_dims=[128, 64]).to(device)

dreamer = DreamerAgent(
    world_model=world_model,
    actor=actor,
    critic=critic,
    imagination_horizon=10,
    gamma=0.99,
    actor_lr=1e-4,
    critic_lr=1e-4,
    device=device
)

print(f"Dreamer Agent created:")
print(f"- Imagination horizon: {dreamer.imagination_horizon}")
print(f"- Discount factor: {dreamer.gamma}")
print(f"- Actor learning rate: {dreamer.actor_lr}")
print(f"- Critic learning rate: {dreamer.critic_lr}")


In [ ]:
print("Testing Dreamer imagination...")

obs, _ = env.reset()
obs_tensor = torch.FloatTensor(obs).to(device)
latent_state = world_model.encode_observations(obs_tensor.unsqueeze(0)).squeeze(0)

imagined_states, imagined_rewards, imagined_actions = dreamer.imagine_trajectory(latent_state, steps=10)

print(f"Imagined {len(imagined_states)} steps")
print(f"Total imagined reward: {sum(imagined_rewards):.2f}")
print(f"Final imagined state shape: {imagined_states[-1].shape}")

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.plot(imagined_rewards, 'g-o', linewidth=2, markersize=4)
plt.title('Imagined Rewards')
plt.xlabel('Imagination Step')
plt.ylabel('Reward')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
imagined_actions = np.array(imagined_actions)
for i in range(min(2, imagined_actions.shape[1])):
    plt.plot(imagined_actions[:, i], label=f'Action {i}', linewidth=2)
plt.title('Imagined Actions')
plt.xlabel('Imagination Step')
plt.ylabel('Action Value')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
imagined_states = np.array(imagined_states)
for i in range(min(4, imagined_states.shape[1])):
    plt.plot(imagined_states[:, i], label=f'Latent {i}', linewidth=1)
plt.title('Imagined Latent States')
plt.xlabel('Imagination Step')
plt.ylabel('Latent Value')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Dreamer Imagination Demo', fontsize=16, y=0.98)
plt.show()


# Section 4: Running Complete Experiments

## 4.1 Using the Experiment Scripts

The modular structure allows running complete experiments with proper training and evaluation:

In [ ]:
"""
from experiments.world_model_experiment import run_world_model_experiment

config = {
    'env_name': 'continuous_cartpole',
    'latent_dim': 32,
    'vae_hidden_dims': [128, 64],
    'dynamics_hidden_dims': [128, 64],
    'reward_hidden_dims': [64, 32],
    'stochastic_dynamics': True,
    'learning_rate': 1e-3,
    'batch_size': 64,
    'train_steps': 1000,
    'data_collection_steps': 5000,
    'data_collection_episodes': 20,
    'rollout_steps': 50
}

world_model, trainer = run_world_model_experiment(config)
"""

print("💡 Experiment scripts are available in the experiments/ directory:")
print("- world_model_experiment.py: Train world models")
print("- rssm_experiment.py: Train RSSM models") 
print("- dreamer_experiment.py: Train complete Dreamer agents")
print("\n📊 Each experiment includes comprehensive evaluation and visualization.")


# Section 5: Key Benefits of Modular Design

## 5.1 Advantages of the Restructured Code

The modular approach provides several benefits:

1. **Reusability**: Components can be imported and used independently
2. **Maintainability**: Clear separation of concerns and organized code
3. **Testability**: Individual components can be tested in isolation
4. **Extensibility**: Easy to add new models, environments, or agents
5. **Collaboration**: Multiple developers can work on different modules

## 5.2 Project Structure Summary

```
CA11/
├── world_models/     # Core model components
├── agents/          # RL agents
├── environments/    # Custom environments
├── utils/           # Utilities and tools
├── experiments/     # Complete training scripts
└── CA11.ipynb       # This demonstration notebook
```

This structure transforms a monolithic notebook into a professional, maintainable codebase suitable for research and development.

In [ ]:
print("🎉 Modular restructuring completed!")
print("\n📚 Key achievements:")
print("✅ Extracted 2000+ lines of code into organized modules")
print("✅ Created reusable world model components")
print("✅ Implemented complete Dreamer agent system")
print("✅ Added comprehensive visualization tools")
print("✅ Developed experiment scripts for systematic evaluation")
print("\n🚀 The modular codebase is now ready for advanced model-based RL research!")
